In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._


In [25]:
%%html

<style>
/*Disable code output line wrapping*/
div.jp-OutputArea-output pre {
    white-space: pre
}
</style>

In [3]:
  val spark = SparkSession.builder()
    .appName("E-commerce Transaction Processing")
    .master("local[*]")
    .getOrCreate()

24/07/26 20:16:18 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@53679695


In [5]:
  val schema = StructType(Array(
    StructField("customer_id", StringType, true),
    StructField("delivery_info", StructType(Array(
      StructField("address", StructType(Array(
        StructField("city", StringType, true),
        StructField("state", StringType, true),
        StructField("street", StringType, true),
        StructField("zip", StringType, true)
      )), true),
      StructField("delivery_time", StringType, true),
      StructField("status", StringType, true)
    )), true),
    StructField("items", ArrayType(StructType(Array(
      StructField("item_id", StringType, true),
      StructField("price", DoubleType, true),
      StructField("product_name", StringType, true),
      StructField("quantity", LongType, true)
    )), true), true),
    StructField("transaction_id", StringType, true),
    StructField("transaction_time", StringType, true)
  ))

schema: org.apache.spark.sql.types.StructType = StructType(StructField(customer_id,StringType,true),StructField(delivery_info,StructType(StructField(address,StructType(StructField(city,StringType,true),StructField(state,StringType,true),StructField(street,StringType,true),StructField(zip,StringType,true)),true),StructField(delivery_time,StringType,true),StructField(status,StringType,true)),true),StructField(items,ArrayType(StructType(StructField(item_id,StringType,true),StructField(price,DoubleType,true),StructField(product_name,StringType,true),StructField(quantity,LongType,true)),true),true),StructField(transaction_id,StringType,true),StructField(transaction_time,StringType,true))


In [10]:
  val rawData = spark.read
//.schema(schema)
    .option("multiline", "true")
    .json("C:\\Users\\paras\\OneDrive\\Desktop\\Startup\\Repositories\\scalaRepos\\complete_big_data_project\\datamigration_project\\src\\main\\scala\\com\\its\\youtube\\chapter1\\datasets")

rawData: org.apache.spark.sql.DataFrame = [customer_id: string, delivery_info: struct<address: struct<city: string, state: string ... 2 more fields>, delivery_time: string ... 1 more field> ... 3 more fields]


In [13]:
rawData.show()

+-----------+--------------------+--------------------+--------------+--------------------+
|customer_id|       delivery_info|               items|transaction_id|    transaction_time|
+-----------+--------------------+--------------------+--------------+--------------------+
|       C006|{{Metropolis, TX,...|[{I013, 1420.15, ...|          T010|2024-07-26T12:54:20Z|
|       C010|{{Star City, TX, ...|[{I011, 366.81, K...|          T011|2024-07-26T12:51:20Z|
|       C009|{{Central City, N...|[{I003, 278.87, L...|          T012|2024-07-26T13:10:20Z|
|       C007|{{Metropolis, NJ,...|[{I018, 1288.94, ...|          T007|2024-07-26T13:26:18Z|
|       C008|{{Star City, NY, ...|[{I002, 285.09, L...|          T008|2024-07-26T13:34:18Z|
|       C009|{{Star City, TX, ...|[{I007, 670.77, L...|          T009|2024-07-26T13:08:18Z|
|       C009|{{Metropolis, TX,...|[{I017, 1073.68, ...|          T001|2024-07-26T12:34:13Z|
|       C003|{{Star City, NJ, ...|[{I003, 1414.45, ...|          T002|2024-07-26

In [23]:
val addressSchema = new StructType()
  .add("street", StringType)
  .add("city", StringType)
  .add("state", StringType)
  .add("zip", StringType)

addressSchema: org.apache.spark.sql.types.StructType = StructType(StructField(street,StringType,true),StructField(city,StringType,true),StructField(state,StringType,true),StructField(zip,StringType,true))


In [35]:
val deliveryAddress = rawData
.withColumn("delivery_info_json", to_json(col("delivery_info")))
.select("transaction_id","customer_id","delivery_info_json")
.select(
    col("*"),
    json_tuple(col("delivery_info_json"), "status", "delivery_time", "address").as(Seq("delivery_status", "delivery_time", "address_json"))
  )
.select("transaction_id", "customer_id", "delivery_status", "delivery_time", "address_json")
.select(
    col("*"),
    json_tuple(col("address_json"), "city", "state", "street", "zip").as(Seq("city", "state", "street", "zip"))
  )
.select("transaction_id", "customer_id", "delivery_status", "delivery_time", "city", "state", "street", "zip")
.show(false)

+--------------+-----------+---------------+--------------------+------------+-----+------------+-----+
|transaction_id|customer_id|delivery_status|delivery_time       |city        |state|street      |zip  |
+--------------+-----------+---------------+--------------------+------------+-----+------------+-----+
|T010          |C006       |Pending        |2024-07-27T13:57:20Z|Metropolis  |TX   |647 Elm St  |43720|
|T011          |C010       |Delivered      |2024-07-30T13:57:20Z|Star City   |TX   |591 Maple St|41627|
|T012          |C009       |Shipped        |2024-07-28T13:57:20Z|Central City|NJ   |541 Elm St  |82347|
|T007          |C007       |Delivered      |2024-07-27T13:57:18Z|Metropolis  |NJ   |760 Pine St |45113|
|T008          |C008       |Pending        |2024-07-29T13:57:18Z|Star City   |NY   |913 Elm St  |72383|
|T009          |C009       |Shipped        |2024-07-29T13:57:18Z|Star City   |TX   |573 Elm St  |61206|
|T001          |C009       |Pending        |2024-07-29T13:57:13Z

deliveryData: Unit = ()


In [36]:
rawData.show()

+-----------+--------------------+--------------------+--------------+--------------------+
|customer_id|       delivery_info|               items|transaction_id|    transaction_time|
+-----------+--------------------+--------------------+--------------+--------------------+
|       C006|{{Metropolis, TX,...|[{I013, 1420.15, ...|          T010|2024-07-26T12:54:20Z|
|       C010|{{Star City, TX, ...|[{I011, 366.81, K...|          T011|2024-07-26T12:51:20Z|
|       C009|{{Central City, N...|[{I003, 278.87, L...|          T012|2024-07-26T13:10:20Z|
|       C007|{{Metropolis, NJ,...|[{I018, 1288.94, ...|          T007|2024-07-26T13:26:18Z|
|       C008|{{Star City, NY, ...|[{I002, 285.09, L...|          T008|2024-07-26T13:34:18Z|
|       C009|{{Star City, TX, ...|[{I007, 670.77, L...|          T009|2024-07-26T13:08:18Z|
|       C009|{{Metropolis, TX,...|[{I017, 1073.68, ...|          T001|2024-07-26T12:34:13Z|
|       C003|{{Star City, NJ, ...|[{I003, 1414.45, ...|          T002|2024-07-26

In [51]:
val itemdf = rawData.select("transaction_id", "items")

itemdf: org.apache.spark.sql.DataFrame = [transaction_id: string, items: array<struct<item_id:string,price:double,product_name:string,quantity:bigint>>]


In [52]:
val itemdesc = itemdf.withColumn("items_description", explode(col("items"))).select("transaction_id", "items_description")

itemdesc: org.apache.spark.sql.DataFrame = [transaction_id: string, items_description: struct<item_id: string, price: double ... 2 more fields>]


In [53]:
itemdesc
.withColumn("items_to_json", to_json(col("items_description")))
.select(
col("*"),
    json_tuple(col("items_to_json"), "item_id", "price", "product_name", "quantity").as(Seq("item_id", "price", "product_name", "quantity"))
  )
.select("transaction_id", "item_id", "price", "product_name", "quantity")
.show()

+--------------+-------+-------+------------+--------+
|transaction_id|item_id|  price|product_name|quantity|
+--------------+-------+-------+------------+--------+
|          T010|   I013|1420.15|     Monitor|       2|
|          T010|   I015| 457.68|       Mouse|       5|
|          T010|   I007|  665.8|       Mouse|       4|
|          T010|   I011|1261.79|     Printer|       1|
|          T010|   I016|1215.65|      Laptop|       1|
|          T011|   I011| 366.81|    Keyboard|       3|
|          T011|   I012| 886.75|     Printer|       4|
|          T011|   I013| 707.81|     Printer|       1|
|          T012|   I003| 278.87|      Laptop|       5|
|          T012|   I001| 230.12|     Monitor|       1|
|          T012|   I011|1462.52|       Mouse|       2|
|          T012|   I019| 909.23|    Keyboard|       2|
|          T007|   I018|1288.94|    Keyboard|       4|
|          T008|   I002| 285.09|      Laptop|       4|
|          T008|   I015|  822.2|    Keyboard|       4|
|         